In [1]:
import os
import numpy as np
import random as rn
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
rn.seed(7)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

from keras import backend as K

tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

import glob
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from keras.models import Model
from keras.layers import Dense, Activation, Input, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU, PReLU

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

Using TensorFlow backend.


In [2]:
folder = "stack2nd/"
oof = pd.read_feather("oof2_all_test.feather")
test = pd.read_feather("test2_all_test.feather")
traindex = oof.item_id

x = oof.drop(["item_id", "y"], axis=1).values
y = oof.y.values
test_id = test.item_id.values
test_x = test.drop(["item_id"], axis=1).values
n_path = x.shape[1]

In [3]:
# result = []
pred_test = np.zeros((len(test_x)))
train_roc, val_roc = [], []
ans = pd.DataFrame(columns=["item_id", "deal_probability"])
early = EarlyStopping(monitor="val_loss", mode="min", patience=1)
folds = pd.read_csv("data/cvFolds.csv")

for j in range(1, 8):
    train_idx = folds[folds.fold!=j].index.values
    test_idx = folds[folds.fold==j].index.values
    xtr = x[train_idx]
    ytr = y[train_idx]
    xval = x[test_idx]
    yval = y[test_idx]
    val_id = traindex[test_idx]
    

    inp_ = Input(shape=[n_path])
    nn = Dense(50)(inp_)
    nn = BatchNormalization()(nn)
    nn = Activation('relu')(nn)
    nn = Dense(16)(nn)
    nn = BatchNormalization()(nn)
    nn = Activation('relu')(nn)
    nn = Dense(1,activation="sigmoid")(nn)
    model = Model(inp_, nn)
    model.compile(optimizer='Adam', loss=root_mean_squared_error)
    
    file_path = "model.hdf5"
    ckpt = ModelCheckpoint(file_path, monitor='val_loss', verbose=2, save_best_only=True, mode='min')
    model.fit(xtr, ytr,batch_size=512*3, epochs=20, verbose=2, validation_data=(xval, yval), callbacks=[ckpt, early])
    model.load_weights(file_path)
    
    pred_val = model.predict(xval)
    pred_val = np.clip(pred_val, 0.0, 1.0)
    loss_val = np.sqrt(mean_squared_error(yval, pred_val))
    
    pred_test += np.clip(model.predict(test_x), 0.0, 1.0).reshape(-1)
    val_roc.append(loss_val)
    
    ans_ = pd.DataFrame({"item_id": val_id, "deal_probability": pred_val.reshape(-1)})
    ans = pd.concat((ans, ans_), axis=0)
    
    print("test loss", loss_val)

print("mean valloss", np.mean(val_roc))

ans.to_csv(folder+"nn1_cvPreds_all.csv", index=False)

sub = pd.DataFrame()
sub["item_id"] = test_id
sub["deal_probability"] = pred_test/7
sub["deal_probability"] .clip(0.0, 1.0, inplace=True)
sub.to_csv(folder+"nn1_testPreds_all.csv", index=False)

Train on 1288649 samples, validate on 214775 samples
Epoch 1/20
 - 13s - loss: 0.2253 - val_loss: 0.2105

Epoch 00001: val_loss improved from inf to 0.21047, saving model to model.hdf5
Epoch 2/20
 - 9s - loss: 0.2093 - val_loss: 0.2096

Epoch 00002: val_loss improved from 0.21047 to 0.20958, saving model to model.hdf5
Epoch 3/20
 - 7s - loss: 0.2092 - val_loss: 0.2092

Epoch 00003: val_loss improved from 0.20958 to 0.20921, saving model to model.hdf5
Epoch 4/20
 - 7s - loss: 0.2091 - val_loss: 0.2087

Epoch 00004: val_loss improved from 0.20921 to 0.20871, saving model to model.hdf5
Epoch 5/20
 - 7s - loss: 0.2091 - val_loss: 0.2087

Epoch 00005: val_loss improved from 0.20871 to 0.20869, saving model to model.hdf5
Epoch 6/20
 - 7s - loss: 0.2090 - val_loss: 0.2092

Epoch 00006: val_loss did not improve from 0.20869
test loss 0.20875820205217824
Train on 1288649 samples, validate on 214775 samples
Epoch 1/20
 - 8s - loss: 0.2168 - val_loss: 0.2099

Epoch 00001: val_loss improved from i